# Summarization

In [1]:
!pip install -U transformers
!pip install -U datasets
!pip install -U sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 10.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0


In [5]:
from datasets import load_dataset

dataset = load_dataset("autopilot-ai/Gujarati-Grammarly-Datasets",data_files=["sentence-pairs/gujarati_sentences_1M.csv"], split = "train[200000:300001]")

In [6]:
dataset = dataset.train_test_split(test_size=0.01)

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Correct', 'Incorrect'],
        num_rows: 99000
    })
    test: Dataset({
        features: ['Correct', 'Incorrect'],
        num_rows: 1001
    })
})

In [8]:
def add_token(row):
    for i in range (len(row["Incorrect"])):
        row["Incorrect"][i] = "<gec>"+row["Incorrect"][i]
        
    return row
dataset_train = dataset["train"].map(add_token, batched=True)
dataset_test = dataset["test"].map(add_token, batched=True)

Map:   0%|          | 0/99000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1001 [00:00<?, ? examples/s]

In [9]:
# dataset_train["Incorrect"][:10]
print(dataset_test["Incorrect"][3])
print(dataset_test["Correct"][3])

<gec>જોકે, હવા ozonator ડૌકતરૌએ જે ચૈટવણિ વિશે સમીક્ષાઓ, ખૂબ કાળજીપૂર્વક સૂચનો અનુસાર ઉપયોગ કરવો જોઇએ.
જોકે, હવા ozonator ડોક્ટરોએ જે ચેતવણી વિશે સમીક્ષાઓ, ખૂબ કાળજીપૂર્વક સૂચનો અનુસાર ઉપયોગ કરવો જોઇએ.


In [13]:
# dataset["train"]["Incorrect"]

In [4]:
# num_samples = len(dataset)
# samples = int(num_samples * 0.7)
# # dataset = dataset_train.take(samples)

# from datasets import Dataset
# sliced_dataset = dataset['train'][0:35000]
# sliced_dataset_valid=dataset['valid'][0:13000]
# # dataset_train=dataset_train[:samples]
# # dataset_train = Dataset.from_dict(sliced_dataset)
# # dataset_valid=Dataset.from_dict(sliced_dataset_valid)

# dataset_train = dataset_train
# dataset_valid=dataset_valid

In [5]:
# import random
# def show_random(sample=3,seed=42):
#     random_index=random.randint(0, len(dataset_train)-5)

#     for i in range(sample):
#         print("\nHeadline\n",dataset['train']['headline'][random_index+i],"\n")
#         print("\nsummary\n",dataset['train']['summary'][random_index+i])
#         # print("article\n",dataset['train']['article'][random_index+i])

# show_random()

In [6]:
!pip install sentencepiece


In [7]:
# from transformers import AutoTokenizer

# checkpoint = "bert-base-uncased"
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [8]:
# tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
#     columns=["attention_mask", "input_ids", "token_type_ids"],
#     label_cols=["input_ids"],
#     shuffle=True,
#     collate_fn=data_collator,
#     batch_size=8,
# )

# tf_validation_dataset = tokenized_datasets["valid"].to_tf_dataset(
#     columns=["attention_mask", "input_ids", "token_type_ids"],
#     label_cols=["input_ids"],
#     shuffle=False,
#     collate_fn=data_collator,
#     batch_size=8,
# )

#Facebook bart

In [7]:
from transformers import AutoTokenizer
import tensorflow as tf
from transformers import TFAutoModelForSeq2SeqLM
from transformers import create_optimizer

model_checkpoint = "Jayveersinh-Raj/mpt5s-guj-grammar-2"
# Create a MirroredStrategy to use multiple GPUs
strategy = tf.distribute.MirroredStrategy()
num_train_epochs = 1
num_train_steps = 197999 * num_train_epochs
model_name = model_checkpoint.split("/")[-1]
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)  # Adjust learning rate as needed
# loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
# metrics = ['rouge']  # Add appropriate metrics here
# early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

# # Define and compile your text generation model within the strategy scope
with strategy.scope():
  tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
  model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
  optimizer, schedule = create_optimizer(
    init_lr=5.6e-5,
    num_warmup_steps=100,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
    )

  model.compile(optimizer=optimizer)

/opt/conda/lib/python3.10/site-packages/keras/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFMT5ForConditionalGeneration.

All the layers of TFMT5ForConditionalGeneration were initialized from the model checkpoint at Jayveersinh-Raj/mpt5s-guj-grammar-2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMT5ForConditionalGeneration for predictions without further training.


In [8]:
# Adding the prefix  as a special token
LANG_TOKEN_MAPPING = {
    'correct sentence': '<gec>'
}
special_tokens_dict = {'additional_special_tokens': list(LANG_TOKEN_MAPPING.values())}
special_tokens_dict

{'additional_special_tokens': ['<gec>']}

In [10]:
# # Adding the special tokens to the tokenizer
# tokenizer.add_special_tokens(special_tokens_dict)

# display all special tokens
tokenizer.all_special_tokens

['</s>', '<unk>', '<pad>', '<gec>']

In [10]:
max_input_length = 128
max_target_length = 128

def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["Incorrect"],
        max_length=max_input_length,
        truncation=True,
        padding=True
    )
    labels = tokenizer(
        examples["Correct"], max_length=max_target_length, truncation=True,padding=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [11]:
dataset_train = dataset_train.map(preprocess_function, batched=True)
dataset_test = dataset_test.map(preprocess_function, batched=True)

Map:   0%|          | 0/99000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1001 [00:00<?, ? examples/s]

In [10]:
from huggingface_hub import notebook_login

notebook_login()

In [11]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [12]:
tf_train_dataset = model.prepare_tf_dataset(
    dataset_train,
    collate_fn=data_collator,
    shuffle=True,
    batch_size=64
)
tf_test_dataset = model.prepare_tf_dataset(
    dataset_test,
    collate_fn=data_collator,
    shuffle=False,
    batch_size=64
)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [13]:
from transformers import create_optimizer
import tensorflow as tf

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

In [17]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Sun Sep  3 14:06:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/

In [1]:
from numba import cuda
cuda.select_device(1)
cuda.close()
cuda.select_device(1)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


<weakproxy at 0x7983c58075b0 to Device at 0x7983c5815780>

In [14]:
import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0, 1"
from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(
    output_dir=f"{model_name}-3", tokenizer=tokenizer
)

model.fit(

    tf_train_dataset,validation_data=tf_test_dataset,callbacks=[callback], epochs=1
)

Cloning https://huggingface.co/Jayveersinh-Raj/mpt5s-guj-grammar-2-3 into local empty directory.


1546/1546 [==============================] - ETA: 0s - loss: 0.0777

Adding files tracked by Git LFS: ['tokenizer.json']. This may take a bit of time if the files are large.


1546/1546 [==============================] - 3032s 2s/step - loss: 0.0777 - val_loss: 0.0375


In [1]:
dataset["test"][1]

NameError: name 'dataset' is not defined

# Inference

In [2]:
from transformers import AutoTokenizer
import tensorflow as tf
from transformers import TFAutoModelForSeq2SeqLM
from transformers import create_optimizer

model_checkpoint = "Jayveersinh-Raj/mpt5s-guj-grammar-2-3"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

/opt/conda/lib/python3.10/site-packages/keras/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFMT5ForConditionalGeneration.

All the layers of TFMT5ForConditionalGeneration were initialized from the model checkpoint at Jayveersinh-Raj/mpt5s-guj-grammar-2-3.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMT5ForConditionalGeneration for predictions without further training.


In [10]:
sent="થોડાના વેગલ એસ્ટેટ વિસ્તારમાં શથાનીક બસમાં તોડફોડ કરવામાં આવી છે."
inputs = tokenizer.encode(sent, return_tensors='tf')
output_ids = model.generate(inputs, max_length=128, num_beams = 4, early_stopping=True)
output = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Print the generated summary
print("Generated Correction:")
print(output)

Generated Correction:
થોડાના વેગલ એસ્ટેટ વિસ્તારમાં શથાનીક બસમાં તોડફડ કરવામાં આવી છે.


In [ ]:
a='भारतकल के तेजी से बदलते वैज्ञानिक प्रसारों में, मानव ग्रीष्मकालीन ऊष्मा का अध्ययन एक महत्वपूर्ण और रोचक क्षेत्र बन चुका है। वैज्ञानिक प्रसारों में, मानव ग्रीष्मकालीन ऊष्मा का अध्ययन एक महत्वपूर्ण'
len(a)

# Evaluation metric rouge

In [3]:
!pip install evaluate
!pip install rouge_score
import nltk
import evaluate
nltk.download('punkt')


rouge_score = evaluate.load("rouge")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=50258fbea530127f4e1aef3957e3e481d79e9892e53285a06ab484effaf88a66
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
from tqdm import tqdm
import numpy as np
from nltk.tokenize import sent_tokenize
from transformers import DataCollatorForSeq2Seq


generation_data_collator = DataCollatorForSeq2Seq(
    tokenizer, model=model, return_tensors="tf"
)

tf_eval_dataset = model.prepare_tf_dataset(
    dataset_test,
    collate_fn=generation_data_collator,
    shuffle=False,
    batch_size=32,
    drop_remainder=True,
)

@tf.function(jit_compile=True)
def generate_with_xla(batch):
    return model.generate(
        input_ids=batch["input_ids"],
        attention_mask=batch["attention_mask"],
        max_new_tokens=128,
    )


all_preds = []
all_labels = []
all_incorrect = []
for batch, labels in tqdm(tf_eval_dataset ):
    predictions = generate_with_xla(batch)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = labels.numpy()
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    incorrect = tokenizer.batch_decode(batch["input_ids"], skip_special_tokens=True)
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    decoded_incorrect = ["\n".join(sent_tokenize(inc.strip())) for inc in incorrect]
    all_preds.extend(decoded_preds)
    all_labels.extend(decoded_labels)
    all_incorrect.extend(decoded_incorrect)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
100%|██████████| 31/31 [01:00<00:00,  1.94s/it]


In [13]:
import pandas as pd
df = pd.DataFrame()
df["actual_correct"] = decoded_labels
df["incorrect"] = decoded_incorrect
df["generated"] = decoded_preds
# print(decoded_incorrect[1])
# print(decoded_preds[1])
# print(decoded_labels[1])
df

,actual_correct,incorrect,generated
0,પરંતુ અસર નિરાશ હોઈ શકે તે પહેલાં એક પુનરાવર્ત...,પરંતુ અસર નિરાશ હૌઇ શકે તે પહૈલાઁ એક પુનરાવર્ત...,પરંતુ અસર નિરાશ હોઈ શકે તે પહેલાં એક પુનરાવર્ત...
1,ન ગમતી ઘટના સાથે સંકળાયેલ લાગણી / વિચાર,ન ગમતી ગતના સાથે શઁકળાયૈલ લાગણી / વિચાર,ન ગમતી ઘટના સાથે સંકળાયેલી લાગણી / વિચાર
2,મીઠું અને મરી ચટણી સાથે ઋતુ.,મિઠૂઁ અને મરી ચટણી સાથે ઋતુ.,મિઠું અને મરી ચટણી સાથે ઋતુ.
3,જ્યાં મુસ્લિમ વસ્તી ૨૫.૩૪ કરોડ રહેશે જે અત્યાર...,જ્યાં મુસ્લિમ વસ્તી ૨૫.૩૪ કરોડ રહેશે જૈ અત્યાર...,જ્યાં મુસ્લિમ વસ્તી ૨૫.૩૪ કરોડ રહેશે જે અત્યાર...
4,"આ easels અને રમકડાં, સ્લાઇડ્સ અને સોફ્ટ ડિઝાઇન...","આ easels અનૈ રમકડાઁ, શલાઈડશ અને સોફ્ટ ડિઝાઇનર્...","આ easels અને રમકડાં, સ્ટાઇડ્સ અને સોફ્ટ ડિઝાઇન..."
5,હવે આ દર ઘટીને 8.65 ટકા થયા છે.,હવે આ દર ઘટીને 8.65 ટકા થયા છે.,હવે આ દર ઘટીને 8.65 ટકા થયા છે.
6,"જેઓ વિવિધ પદ્ધતિઓ શોષણ સંલગ્ન, ક્યારેક ચોક્કસ ...","જેઓ વિવિધ પદધટીઓ શોષણ સંલગ્ન, ક્યારેક ચોક્કસ સ...","જેઓ વિવિધ પદ્ધતિઓ શોષણ સંલગ્ન, ક્યારેક ચોક્કસ ..."
7,5.\nકમ્પ્યુટરમાં ડેટા ભૂલને શુ કહે છે?,5.\nકમ્પ્યુટરમાં ડેટા ભૂલને સૂ કહૈ છે?,5.\nકમ્પ્યુટરમાં ડેટા ભૂલને સૂ કહે છે?
8,બનાવની જાણ થતા જ મોટી સંખ્યામાં લોકો અહી એકત્ર...,બનાવની જાણ થતા જ મોટી સંખ્યામાં લોકો અહી એકત્ર...,બનાવની જાણ થતા જ મોટી સંખ્યામાં લોકો અહી એકત્ર...
9,(યૂનિસેક્સ ટ્રેન્ડ અને ફેશન એટલે એવી વસ્તુઓ તથ...,(યૂનિસેક્સ તરૈનડ અને ફૈસન એટલે એવિ વસ્તુઓ તથા ...,(યૂનિસેક્સ ટ્રેન્ડ અને ફેસન એટલે એવી વસ્તુઓ તથ...


In [16]:
# Function to compare words in two strings and return 1 if all words match, 0 otherwise
def word_match(row):
    actual_correct_words = set(row['actual_correct'].split())
    generated_words = set(row['generated'].split())
    return 1 if actual_correct_words == generated_words else 0

# Apply the word_match function to each row and store the result in a new column 'word_match'
df['word_match'] = df.apply(word_match, axis=1)

# Calculate the average of the 'word_match' column
average_word_match = df['word_match'].mean()
print("Average Word Match:", average_word_match)

Average Word Match: 0.53125


In [17]:
from rouge_score import rouge_scorer
import pandas as pd

# Initialize the ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

rouge_scores = []

# Iterate through rows and calculate ROUGE scores
for index, row in df.iterrows():
    reference = row['actual_correct']  # Ground truth reference
    prediction = row['generated']  # Model-generated prediction

    # Calculate ROUGE scores
    scores = scorer.score(reference, prediction)

    rouge_scores.append(scores)

# Create a new DataFrame to store ROUGE scores
rouge_df = pd.DataFrame(rouge_scores)
# rouge_df.to_csv("bloom-7b/data/rouge_scores.csv", index = False)

In [18]:
rouge1 = list(rouge_df["rouge1"])
rouge2 = list(rouge_df["rouge2"])
rougeL = list(rouge_df["rougeL"])
def mean_rouge(ls : list):
    sum = 0
    count = 0
    for scores in ls:
        count+=1
        sum+=scores[-1]
    return sum/count
avg_rouge1 = mean_rouge(rouge1)
avg_rouge2 = mean_rouge(rouge2)
avg_rougeL = mean_rouge(rougeL)

In [19]:
avg_rouge1, avg_rouge2, avg_rougeL

(0.21875, 0.125, 0.21875)

In [ ]:
# inputs = tokenizer("")
# inputs

hindi_text = "यह एक उदाहरण हिंदी पाठ है।"
tokens = tokenizer.encode(hindi_text, add_special_tokens=True)
tokens

In [ ]:
tokenizer.decode(tokens)

In [27]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model = T5ForConditionalGeneration.from_pretrained("Unbabel/gec-t5_small")
tokenizer = T5Tokenizer.from_pretrained('t5-small')

sentence = "સુંદરકાંડના પ્રારંભમાં હનૂમાન બળવાન તો છે પણ સાથે-સાથે બુદ્ધિમાન પણ છે તેની રોચક ધર્મકથા છૈ"
tokenized_sentence = tokenizer('gec: ' + sentence, max_length=128, truncation=True, padding='max_length', return_tensors='pt')
corrected_sentence = tokenizer.decode(
    model.generate(
        input_ids = tokenized_sentence.input_ids,
        attention_mask = tokenized_sentence.attention_mask, 
        max_length=128,
        num_beams=5,
        early_stopping=True,
    )[0],
    skip_special_tokens=True, 
    clean_up_tokenization_spaces=True
)
print(corrected_sentence) # -> I like swimming.

-


#---------------------------------------------------